# Object-Oriented Programming with Classes
*Caveat: There will be code duplication as classes are built upon to improve functionality. When calling from the REPL, we would need to import the module prior to instantiating.*

### Class Definition
- A class is used to define the structure and behavior of one or more objects
- Each object defined by the class is referred to as an instance of the class
- The class of an object controls its initialization and which attributes and methods are avaialble

In [1]:
# create an empty class
# class names are written in CamelCase
class Flight:
    pass

# create a new object (instance) by calling the class constructor
f = Flight()
print(f)
print(type(f))

<class '__main__.Flight'>


### Instance Methods
- Methods are just functions defined within the class block
- Instance methods are functions which can be called on objects which are instances of our class
- Instance methods must accept a reference to the instance on which the method was called as the first formal argument
- By convention, the first formal argument is *self*

In [2]:
class Flight:

    def flight_number(self):
        return "SN060"

f = Flight()
f.flight_number()

'SN060'

### Instance Initializers
- If provided, the initializer method is called as part of the process of creating a new object when we call the constructor
- The initializer method must be called \_\_init\_\_() delimited by the double underscores (dunder method)
- The first argument to the initializer method must be *self*
- The initializer method should not return anything, it modifies the object referred to by *self*
- Within the initializer we assign to an attribute of the newly created instance called _flight_number
- The leading underscore is used to avoid name clash with the method and as a convention for objects not intended to be consumed or manipulated

In [3]:
class Flight:

    def __init__(self, flight_number):
        self._flight_number = flight_number
    
    def flight_number(self):
        return self._flight_number

f = Flight(flight_number = "SN060")
f.flight_number()

'SN060'

### Validation and Invariants
- It's good practice for the initializer of an object to establish class invariants
- Invariants are truths about the objects of the class that should endure for the lifetime of the object
- We establish class invariants in the \_\_init\_\_() method and raise exceptions if they can't be attained

In [4]:
class Flight:

    def __init__(self, flight_number):
        if not flight_number[:2].isalpha():
            raise ValueError(f"No Airline Code in {flight_number}")
        
        if not flight_number[:2].isupper():
            raise ValueError(f"Invalid Airline Code {flight_number}")
        
        if not (flight_number[2:].isdigit() and int(flight_number[2:]) <= 9999):
            raise ValueError(f"Invalid Route Number {flight_number}")
        
        self._flight_number = flight_number

    def flight_number(self):
        return self._flight_number
    
    def airline(self):
        return self._flight_number[:2]

In [5]:
# testing first check (airline code must be two letters)
# results in ValueError: No Airline Code in 060
# Flight("060")

In [6]:
# testing second check (airline code must be upper case)
# results in ValueError: Invalid Airline Code sn060
# Flight("sn060")

In [7]:
# testing third check (preceeding characters are numeric and less than or equal to 9999)
# results in ValueError: Invalid Route Number SN12345
# Flight('SN12345') 

In [8]:
# flight number passes checks and is returned by flight_number method
# airline code is returned by airline method
f = Flight('SN060')

print(
    f"""
    Airline Code: {f.airline()}
    Flight Number: {f.flight_number()}
    """
)


    Airline Code: SN
    Flight Number: SN060
    


### Second Class
- Model different kinds of aircraft
- The initializer creates four attributes for the aircraft (registration, model, num rows, num seats per row)
- The seating_plan method is added to return the allowed rows and seats as a series of tuples

The call to the range() constructor produces a range object which can be used as an iterable series of row numbers, up to the number of rows in the plane plus one. We add one to the range object because the stopping point of a range is not inclusive. The string and its slice method return a string with one character per seat. These two objects – the range and the string – are bundled up into a tuple.

In [9]:
class Aircraft:

    def __init__(self, registration, model, num_rows, num_seats_per_row):
        self._registration = registration
        self._model = model
        self._num_rows = num_rows
        self._num_seats_per_row = num_seats_per_row
    
    def registration(self):
        return self._registration
    
    def model(self):
        return self._model
    
    def seating_plan(self):
        return (range(1, self._num_rows + 1), "ABCDEFGHJK"[:self._num_seats_per_row])

In [10]:
# construct a plane with a seating plan
a = Aircraft(" G-EUPT", "Airbus A319", num_rows = 22, num_seats_per_row = 6)

print(
    f"""
    Registration: {a.registration()}
    Model: {a.model()}
    Seating Plan: {a.seating_plan()}
    """
)


    Registration:  G-EUPT
    Model: Airbus A319
    Seating Plan: (range(1, 23), 'ABCDEF')
    


In [11]:
# test range object (should include 22 rows and 6 seats per row (letters))
row_seats = []

for item in a.seating_plan():
    for identifier in item:
        row_seats.append(identifier)

%pprint
row_seats

Pretty printing has been turned OFF


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 'A', 'B', 'C', 'D', 'E', 'F']

In [12]:
%pprint

Pretty printing has been turned ON


### Collaborating Classes
The Law of Demeter is an object-oriented design principle that says you should never call methods on objects you receive from other calls. Or, put another way: Only talk to your immediate friends.

We’ll now modify our Flight class to accept an aircraft object when it is constructed, and we’ll follow the Law of Demeter by adding a method to report the aircraft model. This method will delegate to Aircraft on behalf of the client rather than allowing the client to “reach through” the Flight and interrogate the Aircraft object directly.

Notice that we construct the Aircraft object and directly pass it to the Flight constructor without needing an intermediate named reference for it.

In [13]:
class Flight:
    """A Flight with a Particular Passenger Aircraft"""
    
    def __init__(self, flight_number, aircraft):
        if not flight_number[:2].isalpha():
            raise ValueError(f"No Airline Code in {flight_number}")
        
        if not flight_number[:2].isupper():
            raise ValueError(f"Invalid Airline Code {flight_number}")
        
        if not (flight_number[2:].isdigit() and int(flight_number[2:]) <= 9999):
            raise ValueError(f"Invalid Route Number {flight_number}")
        
        self._flight_number = flight_number
        self._aircraft = aircraft

    def flight_number(self):
        return self._flight_number
    
    def airline(self):
        return self._flight_number[:2]

    def aircraft_model(self):
        return self._aircraft.model()


class Aircraft:

    def __init__(self, registration, model, num_rows, num_seats_per_row):
        self._registration = registration
        self._model = model
        self._num_rows = num_rows
        self._num_seats_per_row = num_seats_per_row
    
    def registration(self):
        return self._registration
    
    def model(self):
        return self._model
    
    def seating_plan(self):
        return (range(1, self._num_rows + 1), "ABCDEFGHJK"[:self._num_seats_per_row])

In [14]:
# construct a flight with a specific aircraft
f = Flight("BA758", Aircraft("G-EUPT", "Airbus A319", num_rows = 22, num_seats_per_row = 6))

f.aircraft_model()

'Airbus A319'

### Booking Seats